In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINConv

# --- 1. RAAG DEFINITION & REDUCTION ---
# Commutation Graph: Even nodes commute with the next odd node
RAAG_ADJ = torch.zeros((2000, 2000))
for i in range(1999):
    if i % 2 == 0:
        RAAG_ADJ[i][i+1] = 1
        RAAG_ADJ[i+1][i] = 1

class RAAGWord:
    def __init__(self, letters, adj_matrix=RAAG_ADJ):
        self.adj_matrix = adj_matrix
        self.letters = self.reduce(letters)

    def reduce(self, lts):
        res = []
        for l in lts:
            if not res:
                res.append(l)
                continue

            # RAAG Reduction: Shuffle past commuting elements to find inverse
            idx = len(res) - 1
            can_cancel = False
            while idx >= 0:
                if res[idx] == -l:
                    can_cancel = True
                    break
                # If 'l' doesn't commute with res[idx], it's blocked from moving further
                if not self.adj_matrix[abs(l)][abs(res[idx])]:
                    break
                idx -= 1

            if can_cancel:
                res.pop(idx)
            else:
                res.append(l)
        return res

    def __mul__(self, other):
        return RAAGWord(self.letters + other.letters, self.adj_matrix)

    def inv(self):
        return RAAGWord([-l for l in reversed(self.letters)], self.adj_matrix)

# --- 2. ALGEBRAIC MOVES & TOPOLOGY ---
def nielsen_move(gens):
    new_gens = list(gens)
    i, j = random.sample(range(len(gens)), 2)
    op = random.choice(['mul', 'inv_mul'])
    if op == 'mul':
        new_gens[i] = (new_gens[i][0] * new_gens[j][0], new_gens[i][1] * new_gens[j][1])
    else:
        j_inv = (new_gens[j][0].inv(), new_gens[j][1].inv())
        new_gens[i] = (new_gens[i][0] * j_inv[0], new_gens[i][1] * j_inv[1])
    return new_gens

def subgroup_to_graph(subgroup):
    x, edge_index = [], []
    # 5 Distinct Hubs (using 501-505 as identifiers)
    for i in range(5): x.append([float(501 + i)])
    curr_idx = 5
    for i, (w_a, w_b) in enumerate(subgroup):
        hub_idx = i
        full_word = w_a.letters + w_b.letters
        if not full_word: full_word = [0]
        for j, letter in enumerate(full_word):
            x.append([float(letter)])
            # Connect letter node to the respective generator hub
            edge_index.append([curr_idx, hub_idx]); edge_index.append([hub_idx, curr_idx])
            # Sequence edges (path topology)
            if j > 0:
                edge_index.append([curr_idx - 1, curr_idx]); edge_index.append([curr_idx, curr_idx - 1])
            curr_idx += 1
    return Data(x=torch.tensor(x, dtype=torch.float),
                edge_index=torch.tensor(edge_index, dtype=torch.long).t().contiguous())

# --- 3. MODEL: UNIVERSAL HUB GNN ---
class UniversalHubGNN(nn.Module):
    def __init__(self, hidden=128):
        super().__init__()
        self.hidden = hidden
        self.embed = nn.Embedding(2001, hidden) # Handle range -1000 to 1000
        self.lin_deg = nn.Linear(1, hidden)
        self.convs = nn.ModuleList([
            GINConv(nn.Sequential(nn.Linear(hidden, hidden), nn.ReLU(), nn.Linear(hidden, hidden)))
            for _ in range(6)
        ])
        self.lns = nn.ModuleList([nn.LayerNorm(hidden) for _ in range(6)])
        # Input to FC is 5 hubs * hidden_size
        self.fc = nn.Linear(5 * hidden, 64)

        # Add this method to your UniversalHubGNN class
        def forward(self, x, edge_index, batch=None, **kwargs):
            from torch_geometric.data import Data
            # Reconstruct the object your forward_one expects
            data = Data(x=x, edge_index=edge_index, batch=batch)
            return self.forward_one(data)

    def forward_one(self, data):
        # Normalizing input indices for embedding
        idx = torch.clamp((data.x.view(-1) + 1000).long(), 0, 2000)
        x = self.embed(idx)

        # Degree-based structural features
        row, _ = data.edge_index
        deg = torch.zeros((data.x.size(0), 1), device=data.x.device)
        deg.scatter_add_(0, row.unsqueeze(1), torch.ones((row.size(0), 1), device=data.x.device))
        x = x + self.lin_deg(deg)

        for conv, ln in zip(self.convs, self.lns):
            h = conv(x, data.edge_index)
            x = ln(F.dropout(F.relu(h), p=0.1, training=self.training) + x)

        # Robust Slot-Aware Pooling
        num_graphs = data.num_graphs if hasattr(data, 'num_graphs') else 1
        hub_embeddings = []
        if hasattr(data, 'ptr') and data.ptr is not None:
            for i in range(num_graphs):
                start = data.ptr[i]
                hub_embeddings.append(x[start : start + 5].reshape(-1))
        else:
            hub_embeddings.append(x[:5].reshape(-1))

        x_final = torch.stack(hub_embeddings)
        return F.normalize(self.fc(x_final), p=2, dim=1)

# --- 4. DATA GENERATION: UPDATED ---
def generate_triplets(num_samples, complexity, neg_type):
    triplets = []
    for _ in range(num_samples):
        shift = random.randint(-400, 400)
        A = [RAAGWord([shift + i]) for i in range(1, 6)]
        B = [RAAGWord([shift + i + 100]) for i in range(1, 6)]
        anchor_sub = [(A[i], B[i]) for i in range(5)]

        # Positive: Scrambled via Nielsen moves
        pos_sub = anchor_sub
        for _ in range(complexity):
            pos_sub = nielsen_move(pos_sub)

        # Negative Logic
        if neg_type == "Alphabet":
            ns = shift + random.choice([-300, 300])
            neg_sub = [(RAAGWord([ns+i]), RAAGWord([ns+i+100])) for i in range(1, 6)]
        elif neg_type == "Identity":
            neg_sub = list(anchor_sub)
            neg_sub[random.randint(0, 4)] = (RAAGWord([]), RAAGWord([]))
        elif neg_type == "Commutator":
            neg_sub = list(anchor_sub)
            i, j = random.sample(range(5), 2)
            g1, g2 = anchor_sub[i], anchor_sub[j]
            # Replace a generator with its commutator
            neg_sub[random.randint(0, 4)] = (
                g1[0]*g2[0]*g1[0].inv()*g2[0].inv(),
                g1[1]*g2[1]*g1[1].inv()*g2[1].inv()
            )
        else: # "Swap" - Surgical change in one slot
            neg_sub = list(anchor_sub)
            idx = random.randint(0, 4)
            neg_sub[idx] = (RAAGWord([shift + 999]), anchor_sub[idx][1])

        triplets.append((
            subgroup_to_graph(anchor_sub),
            subgroup_to_graph(pos_sub),
            subgroup_to_graph(neg_sub)
        ))
    return triplets

def collate(batch):
    return (Batch.from_data_list([b[0] for b in batch]),
            Batch.from_data_list([b[1] for b in batch]),
            Batch.from_data_list([b[2] for b in batch]))

# --- 5. EXECUTION ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
neg_cases = ["Alphabet", "Identity", "Commutator", "Swap"]


all_results = []

models = {}
models = {case: None for case in neg_cases}

for nielsen_moves in range(10, 41, 10):
  final_results = {}
  for case in neg_cases:
      print(f"\n>>> TRAINING SETTING: {case}")
      if models[case] == None:
          model = UniversalHubGNN().to(device)
      else:
          model = models[case]
      optimizer = torch.optim.AdamW(model.parameters(), lr=0.0002)
      criterion = nn.TripletMarginLoss(margin=0.5)

      for epoch in range(500): # Adjust epoch count as needed
          train_loader = DataLoader(generate_triplets(512, nielsen_moves, case), batch_size=16, collate_fn=collate, shuffle=True)
          model.train()
          t_loss = 0
          for a, p, n in train_loader:
              optimizer.zero_grad()
              ea, ep, en = model.forward_one(a.to(device)), model.forward_one(p.to(device)), model.forward_one(n.to(device))
              loss = criterion(ea, ep, en)
              loss.backward(); optimizer.step(); t_loss += loss.item()


          print(f"Epoch {epoch:03d} | Loss: {t_loss/len(train_loader):.4f}")
          if t_loss/len(train_loader) < 0.01:
            break

      # Stress Test
      print(f"Testing {case}...")
      model.eval()
      test_data = generate_triplets(100, nielsen_moves, case)
      p_sims, n_sims = [], []
      with torch.no_grad():
          for a, p, n in test_data:
              ea, ep, en = model.forward_one(a.to(device)), model.forward_one(p.to(device)), model.forward_one(n.to(device))
              p_sims.append(F.cosine_similarity(ea, ep).item())
              n_sims.append(F.cosine_similarity(ea, en).item())

      final_results[case] = {"Pos": np.mean(p_sims), "Neg": np.mean(n_sims)}
      models[case] = model
  all_results.append(final_results)
  torch.save(model, 'full_gin_model.pth')

  # --- FINAL REPORT ---
  print("\n" + "="*50 + f"\nFINAL COMPARISON REPORT {nielsen_moves}\n" + "="*50)
  for case, res in final_results.items():
      print(f"{case:<12} | Scrambled: {res['Pos']:.4f} | Neg: {res['Neg']:.4f} | GAP: {res['Pos']-res['Neg']:.4f}")



>>> TRAINING SETTING: Alphabet
Epoch 000 | Loss: 0.4128
Epoch 001 | Loss: 0.2606
Epoch 002 | Loss: 0.2221
Epoch 003 | Loss: 0.1819
Epoch 004 | Loss: 0.1385
Epoch 005 | Loss: 0.0987
Epoch 006 | Loss: 0.0754
Epoch 007 | Loss: 0.0528
Epoch 008 | Loss: 0.0491
Epoch 009 | Loss: 0.0430
Epoch 010 | Loss: 0.0359
Epoch 011 | Loss: 0.0322
Epoch 012 | Loss: 0.0293
Epoch 013 | Loss: 0.0255
Epoch 014 | Loss: 0.0195
Epoch 015 | Loss: 0.0167
Epoch 016 | Loss: 0.0116
Epoch 017 | Loss: 0.0129
Epoch 018 | Loss: 0.0143
Epoch 019 | Loss: 0.0119
Epoch 020 | Loss: 0.0093
Testing Alphabet...

>>> TRAINING SETTING: Identity
Epoch 000 | Loss: 0.3013
Epoch 001 | Loss: 0.0008
Testing Identity...

>>> TRAINING SETTING: Commutator
Epoch 000 | Loss: 0.5417
Epoch 001 | Loss: 0.3774
Epoch 002 | Loss: 0.3033
Epoch 003 | Loss: 0.2716
Epoch 004 | Loss: 0.2126
Epoch 005 | Loss: 0.1577
Epoch 006 | Loss: 0.1222
Epoch 007 | Loss: 0.1010
Epoch 008 | Loss: 0.0831
Epoch 009 | Loss: 0.0592
Epoch 010 | Loss: 0.0561
Epoch 011 | 

In [30]:
import torch
import torch.nn.functional as F
import random
import copy

def apply_nielsen_moves_fixed(subgroup_basis, num_moves=50):
    """Scrambles the subgroup while maintaining your RAAGWord reduction logic."""
    scrambled = copy.deepcopy(subgroup_basis)
    rank = len(scrambled)

    for _ in range(num_moves):
        i, j = random.sample(range(rank), 2)
        op = random.choice(['mul', 'inv_mul'])
        
        # We use your existing __mul__ and inv() methods
        if op == 'mul':
            scrambled[i] = (scrambled[i][0] * scrambled[j][0], 
                            scrambled[i][1] * scrambled[j][1])
        else:
            j_inv = (scrambled[j][0].inv(), scrambled[j][1].inv())
            scrambled[i] = (scrambled[i][0] * j_inv[0], 
                            scrambled[i][1] * j_inv[1])
    return scrambled

def execute_committee_crack(models_dict, device, secret_shift=42):
    """
    Runs the crack test across all trained specialist models.
    models_dict: { 'Alphabet': model1, 'Identity': model2, ... }
    """
    # 1. Prepare the Secret Basis and Anchor Graph once
    secret_basis = [(RAAGWord([secret_shift + i]), RAAGWord([secret_shift + i + 100])) for i in range(5)]
    # Use the same vault for every model for a fair comparison
    correct_index = random.randint(0, 99)
    vault = []
    
    print(f"🔒 Hiding secret in Vault Slot: {correct_index}...")
    for i in range(100):
        if i == correct_index:
            vault.append(apply_nielsen_moves_fixed(secret_basis, num_moves=40))
        else:
            r_shift = random.randint(-400, 400)
            if abs(r_shift - secret_shift) < 50: r_shift += 200
            noise_basis = [(RAAGWord([r_shift + j]), RAAGWord([r_shift + j + 100])) for j in range(5)]
            vault.append(apply_nielsen_moves_fixed(noise_basis, num_moves=10))

    # 2. Iterate through each specialist model
    results_summary = {}

    for case_name, model in models_dict.items():
        if model is None: continue
        
        model.eval()
        model.to(device)
        print(f"\n🔎 Testing Specialist: [{case_name}]")
        
        # Calculate Anchor Embedding for this specific model's latent space
        anchor_graph = subgroup_to_graph(secret_basis).to(device)
        with torch.no_grad():
            anchor_emb = model.forward_one(anchor_graph)

        similarities = []
        for sub in vault:
            g = subgroup_to_graph(sub).to(device)
            with torch.no_grad():
                emb = model.forward_one(g)
                sim = F.cosine_similarity(anchor_emb, emb).item()
                similarities.append(sim)

        best_idx = np.argmax(similarities)
        best_sim = similarities[best_idx]
        success = (best_idx == correct_index)
        
        results_summary[case_name] = {
            "success": success,
            "best_sim": best_sim,
            "gap": best_sim - np.mean(similarities) # How much the secret stands out
        }
        
        status = "✅ SUCCESS" if success else "❌ FAILURE"
        print(f"   Result: {status} | Sim: {best_sim:.4f} | Found at: {best_idx}")

    # 3. Final Report
    print("\n" + "="*50 + "\nCOMMITTEE CRACK SUMMARY\n" + "="*50)
    for case, data in results_summary.items():
        outcome = "PASS" if data['success'] else "FAIL"
        print(f"{case:<12} : {outcome:<5} | Confidence Gap: {data['gap']:.4f}")
    print("="*50)

# Run the updated crack
execute_committee_crack(models, device, secret_shift=42)

🔒 Hiding secret in Vault Slot: 76...

🔎 Testing Specialist: [Alphabet]
   Result: ❌ FAILURE | Sim: 0.7938 | Found at: 52

🔎 Testing Specialist: [Identity]
   Result: ❌ FAILURE | Sim: 0.9817 | Found at: 29

🔎 Testing Specialist: [Commutator]
   Result: ❌ FAILURE | Sim: 0.9923 | Found at: 5

🔎 Testing Specialist: [Swap]
   Result: ❌ FAILURE | Sim: 0.9875 | Found at: 93

COMMITTEE CRACK SUMMARY
Alphabet     : FAIL  | Confidence Gap: 0.4442
Identity     : FAIL  | Confidence Gap: 0.0117
Commutator   : FAIL  | Confidence Gap: 0.4612
Swap         : FAIL  | Confidence Gap: 0.0048
